In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import interp1d
from pathlib import Path
import math

In [11]:
class MassTrack:
    def __init__(data):
        global model

    def get_ZAMS(data):
        global ZAMS_array
        try:
            cut_array = data[data[:,28]> data[1,28] - 0.0015]
            Lfus = np.sum(cut_array[:, 13:17], axis = 1)
            Ltot = (10**(cut_array[:, 3]))* (3.846 * 10**33)
            LfusdivLtot = np.divide(Lfus, Ltot)
            ZAMS_age = cut_array[0:LfusdivLtot.argmax(), 2]
            LfusdivLtot_lim = LfusdivLtot[0:LfusdivLtot.argmax()]
            #plt.plot(LfusdivLtot_lim, ZAMS_age)

            ZAMS_x = LfusdivLtot_lim
            ZAMS_y = ZAMS_age

            int_curve = interp1d(ZAMS_x, ZAMS_y)
            interp_age = int_curve(0.999)
            int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            ZAMS = np.array(int_curve_two(interp_age))
            return ZAMS

        except:
            print("followed except")
            central_temperature = data[:,23]
            ZAMS = data[central_temperature.argmax(),:]
            ##ZAMS_array = np.row_stack((ZAMS_array, ZAMS))
            return ZAMS

    def get_IAMS(data):
        global IAMS_array
        try:
            IAMS_x = data[:,28]
            IAMS_y = data[:,2]
            IAMS_int_curve = interp1d(IAMS_x, IAMS_y)
            IAMS_age = IAMS_int_curve(0.3)
            IAMS_int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            IAMS = np.array(IAMS_int_curve_two(IAMS_age))
            return IAMS
            ##IAMS_array = np.row_stack((IAMS_array, IAMS))
            ##print("good")

        except:
            IAMS_error = np.zeros(shape = [1, 61])
            #IAMS_array = np.row_stack((IAMS_array, IAMS_error))
            #print("error")

    def get_TAMS(data):
        global TAMS_array
        try:
            TAMS_x = data[:,28]
            TAMS_y = data[:,2]
            TAMS_int_curve = interp1d(TAMS_x, TAMS_y)
            global TAMS_age 
            TAMS_age = TAMS_int_curve(10e-12)
            TAMS_int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            TAMS = np.array(TAMS_int_curve_two(TAMS_age))
            return TAMS
        except:
            Col = data[0,:]
            ## TAMS_age set to 100 so that when TAMS_age is used to filter data in RGBTip_Function, the program errors and doesnt run for models that dont make it past the TAMS.
            TAMS_age = 100
            End = Col.argmax()
            TAMS = np.array(data[:,End])

    def get_RGBTip(data):
        global RGBTip_array
        try:
            cut_array = data[data[:, 2] > TAMS_age]
            Lum = (10**(cut_array[:, 3]))* (3.846 * 10**33)
            Lum_Max = Lum.argmax()
            Teff = (10**(cut_array[:, 6]))
            Teff_Min = Teff.argmin()
            if (Teff_Min > Lum_Max):
                RGBTip = cut_array[Lum_Max,:]
                return RGBTip
            else:
                RGBTip = cut_array[Teff_Min,:]
                return RGBTip
        except:
            RGBTip_error = np.arange(60)
            RGBTip_error = RGBTip_error.fill(0)

    def get_sEEPs(data):
        global sEEPs
        sEEp_ZAMS = (MassTrack.get_ZAMS(data))
        sEEp_IAMS = (MassTrack.get_IAMS(data))
        sEEp_TAMS = (MassTrack.get_TAMS(data))
        sEEp_RGBTip = (MassTrack.get_RGBTip(data))
        
        #weight values for secondary EEP distribution
        Weight_Val_Age = 4.0  #4.0
        Weight_Val_Lum = 10  #10.0
        Weight_Val_Temp = 1.25  #1.25
        

        ZAMS_weight = math.sqrt((Weight_Val_Age*(sEEp_ZAMS[2])**2)+(Weight_Val_Temp*(sEEp_ZAMS[6])**2)+(Weight_Val_Lum*(sEEp_ZAMS[3])**2))
        IAMS_weight = ZAMS_weight + math.sqrt((Weight_Val_Age*(sEEp_IAMS[2]-sEEp_ZAMS[2])**2)+(Weight_Val_Temp*(sEEp_IAMS[6]-sEEp_ZAMS[6])**2)+(Weight_Val_Lum*(sEEp_IAMS[3]-sEEp_ZAMS[3])**2))
        TAMS_weight = IAMS_weight + math.sqrt((Weight_Val_Age*(sEEp_TAMS[2]-sEEp_IAMS[2])**2)+(Weight_Val_Temp*(sEEp_TAMS[6]-sEEp_IAMS[6])**2)+(Weight_Val_Lum*( sEEp_TAMS[3]-sEEp_IAMS[3])**2))
        RGBTip_weight = TAMS_weight + math.sqrt((Weight_Val_Age*(sEEp_RGBTip[2]-sEEp_TAMS[2])**2)+(Weight_Val_Temp*(sEEp_RGBTip[6]-sEEp_TAMS[6])**2)+(Weight_Val_Lum*(sEEp_RGBTip[3]-sEEp_TAMS[3])**2))

        b = data[data[:,2] > sEEp_ZAMS[2]]
        n=0
        point_weight = np.array([ZAMS_weight])
        
        for row in b:
            if(n < b.shape[0]-1):
                ##find the weight of all plotted points in the mass track
                weight = point_weight[n] + math.sqrt((4.0*(b[n+1,2]-b[n,2])**2)+(1.25*(b[n+1,6]-b[n,6])**2)+(10*(b[n+1,3]-b[n,3])**2))
                point_weight = np.row_stack([point_weight, weight])
                n = n+1
            else:
                n=n

        c = np.column_stack((b,point_weight))
        Dzi = np.linspace(ZAMS_weight, IAMS_weight,201)
        Dtr = np.linspace(TAMS_weight, RGBTip_weight, 201)
        Dit = np.linspace(IAMS_weight, TAMS_weight, 201)

        D_curve = interp1d(point_weight[:,0], c[:,0:], axis=0)
        
        properties_ZtoI = D_curve(Dzi)
        properties_ItoT = D_curve(Dit)
        properties_TtoR = D_curve(Dtr)
        sEEPs = np.concatenate((properties_ZtoI, properties_ItoT, properties_TtoR), axis=0)
        sEEPs = np.delete(sEEPs, (61), axis=1)
        return sEEPs

In [13]:
#Setting up dictionarys for storing track EEP values
models = {}
global models

ZAMS = "ZAMS"
IAMS = "IAMS"
TAMS = "TAMS"
RGBTip = "RGBTip"
sEEP = "sEEP"


#Change folder to directory of tracks
for (root, dirs, file) in os.walk('./Complete_files_folder/'):
    for track in file:
        model =('./Complete_files_folder/' + track)
        data = np.genfromtxt(model, usecols = range(0,61))
        models[track] = {}
        models[track][ZAMS] = MassTrack.get_ZAMS(data)
        models[track][IAMS] = MassTrack.get_IAMS(data)
        models[track][TAMS] = MassTrack.get_TAMS(data)
        models[track][RGBTip] = MassTrack.get_RGBTip(data)
        models[track][sEEP]= MassTrack.get_sEEPs(data)

[[4.30000000e+02 5.31000000e+02 5.99410889e-02 ... 2.73200000e+00
  5.77700000e-03 3.67100000e-07]
 [4.93625302e+02 5.34000000e+02 2.24524807e-01 ... 2.46737470e+00
  5.11500000e-03 4.66812892e-07]
 [5.14410134e+02 5.34000000e+02 3.89298264e-01 ... 2.45000000e+00
  5.08800000e-03 4.89069121e-07]
 ...
 [1.91430598e+03 8.64305976e+02 1.99192691e+01 ... 1.21738805e+00
  5.11424621e-03 3.84188048e-08]
 [1.91942618e+03 8.69000000e+02 1.99355405e+01 ... 1.20914764e+00
  5.06816437e-03 3.73790255e-08]
 [1.92447277e+03 8.73000000e+02 1.99515779e+01 ... 1.20152723e+00
  5.02374511e-03 3.64249022e-08]]
[[4.28000000e+02 5.30000000e+02 5.72593225e-02 ... 3.00000000e+00
  6.31900000e-03 3.99800000e-07]
 [4.90797392e+02 5.33000000e+02 2.10012403e-01 ... 2.72800000e+00
  5.64479739e-03 5.00496088e-07]
 [5.11543872e+02 5.33000000e+02 3.63259424e-01 ... 2.71600000e+00
  5.63700000e-03 5.24343872e-07]
 ...
 [4.64339892e+03 1.38500000e+03 1.91019570e+01 ... 2.52660108e-01
  1.81660108e-04 9.40120217e-09]

[[4.36000000e+02 6.80000000e+02 1.27906102e-02 ... 1.52800000e+01
  2.23100000e-02 1.07900000e-06]
 [4.38775544e+02 6.79000000e+02 1.29757012e-02 ... 1.51789782e+01
  2.19746901e-02 1.06157119e-06]
 [4.42026622e+02 6.76973378e+02 1.32192355e-02 ... 1.50594676e+01
  2.16884027e-02 1.05197338e-06]
 ...
 [4.26171702e+03 1.68128298e+03 2.72898134e+00 ... 3.10328298e-01
  2.59356596e-04 5.88571702e-09]
 [4.36533542e+03 1.69333542e+03 2.72991808e+00 ... 2.97966458e-01
  2.40432917e-04 5.83332917e-09]
 [4.47537349e+03 1.70562651e+03 2.73079638e+00 ... 2.86162651e-01
  2.22962651e-04 5.79437349e-09]]
[[4.39000000e+02 6.80000000e+02 1.10036621e-02 ... 1.68100000e+01
  2.28800000e-02 1.14500000e-06]
 [4.41115570e+02 6.78000000e+02 1.11188326e-02 ... 1.67253772e+01
  2.26172873e-02 1.11995987e-06]
 [4.43458782e+02 6.77000000e+02 1.12581101e-02 ... 1.66462365e+01
  2.23978853e-02 1.10416487e-06]
 ...
 [3.04431866e+03 1.56268134e+03 2.21333523e+00 ... 4.65972536e-01
  5.27945071e-04 6.84436268e-09]